
# Scenario 2 — Health Sensing

This notebook implements the tasks described in *Scenario 2 - Health Sensing*:
- Visualizations for each participant (Nasal Airflow, Thoracic Movement, SpO₂) with event overlays
- Signal cleaning (bandpass filtering to retain breathing frequencies)
- Dataset creation: 30-second windows with 50% overlap and labeling from event annotations
- Model scaffolding for 1D CNN and Conv-LSTM and evaluation using Leave-One-Participant-Out (LOSO) CV


**Notes & assumptions**

- Input data folder structure per participant should follow the assignment convention, e.g.:

```
Data/AP01/
  nasal_airflow.txt
  thoracic_movement.txt
  spo2.txt
  flow_events.csv
  sleep_profile.csv  # optional
```

- The notebook **does not** run long training loops automatically (training cells are provided and can be executed by the user). This keeps the notebook interactive and safe to run on modest hardware.

- For full task text, refer to the provided PDF (Scenario 2 - Health Sensing).

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

# Standard imports
import os, json, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import butter, filtfilt, sosfiltfilt, sosfilt, sosfreqz
from datetime import timedelta

# ML imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# Helper for reproducibility
RND = 42
np.random.seed(RND)
torch.manual_seed(RND)

print('Libraries loaded.')


Libraries loaded.


In [7]:
def load_signal_txt(path, time_col=0, value_col=1, time_format=None):
    """
    Robust loader for breathing signals.
    Handles messy whitespace, extra columns, multiple delimiters, inconsistent rows.
    Always returns a pandas Series with a datetime index.
    """

    # STEP 1 — Load with extremely tolerant settings
    try:
        df = pd.read_csv(
            path,
            sep=r"\s+|,|;|\t",   # any whitespace, comma, semicolon or tab
            engine="python",
            header=None,
            comment="#",
            on_bad_lines="skip"   # <--- prevents ParserError
        )
    except Exception as e:
        raise ValueError(f"Could not read file {path}: {e}")

    # Remove fully empty columns
    df = df.dropna(axis=1, how='all')

    # Ensure enough columns
    if df.shape[1] < max(time_col, value_col) + 1:
        raise ValueError(
            f"File {path} does not contain enough columns. "
            f"Found {df.shape[1]}, but time_col={time_col}, value_col={value_col}"
        )

    # Extract time/value columns
    time_data = df.iloc[:, time_col]
    value_data = df.iloc[:, value_col]

    # STEP 2 — Convert time column
    if pd.api.types.is_numeric_dtype(time_data):
        # treat as seconds offset
        timestamps = pd.to_timedelta(time_data, unit="s")
        index = pd.Timestamp("2000-01-01") + timestamps
    else:
        index = pd.to_datetime(time_data, format=time_format, errors="coerce")

    # STEP 3 — Convert values to float
    values = pd.to_numeric(value_data, errors="coerce")

    # Build Series
    s = pd.Series(values.values, index=index).dropna()

    return s

# Example usage:
# nasal = load_signal_txt('/content/drive/MyDrive/projects/Health_Sensing/Data/AP01/Flow-30-05-2024.txt', time_col=0, value_col=1)


In [14]:

# ------------------------------------------------------------------------------------
# ✔ Robust Signal Loader (handles inconsistent columns, spaces, tabs, commas)
# ------------------------------------------------------------------------------------
def load_signal_txt(path, time_col=0, value_col=1, time_format=None):
    df = pd.read_csv(
        path,
        sep=r"\s+|,|;|\t",
        engine="python",
        header=None,
        comment="#",
        on_bad_lines="skip"
    )

    # Drop empty columns
    df = df.dropna(axis=1, how="all")

    # Extract required columns
    time_data = df.iloc[:, time_col]
    value_data = df.iloc[:, value_col]

    # Convert time column
    if pd.api.types.is_numeric_dtype(time_data):
        index = pd.Timestamp("2000-01-01") + pd.to_timedelta(time_data, unit="s")
    else:
        index = pd.to_datetime(time_data, errors="coerce")

    # Convert values
    values = pd.to_numeric(value_data, errors="coerce")

    # Build series
    s = pd.Series(values.values, index=index).dropna()
    return s


# ------------------------------------------------------------------------------------
# ✔ Robust Event Loader (handles spaces, tabs, extra columns)
# ------------------------------------------------------------------------------------
def load_events(path):
    df = pd.read_csv(
        path,
        sep=r"\s+|,|;|\t",
        engine="python",
        header=None,
        on_bad_lines="skip",
        comment="#"
    )

    # Drop empty columns
    df = df.dropna(axis=1, how="all")

    # Event file usually has 3 columns → start, end, type
    if df.shape[1] < 3:
        raise ValueError(f"Event file {path} must have at least 3 columns.")

    df.columns = ["start_time", "end_time", "event_type"]

    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
    df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")

    return df.dropna()


# ------------------------------------------------------------------------------------
# ✔ Fixed Visualization Function
# ------------------------------------------------------------------------------------
def visualize_participant(part_dir, out_pdf_path=None, show=False):

    p = Path(part_dir)

    nasal_path = p / 'Flow-30-05-2024.txt'
    thor_path  = p / 'Thorac-30-05-2024.txt'
    spo2_path  = p / 'SPO2-30-05-2024.txt'
    events_path = p / 'FlowEvents-30-05-2024.txt'

    # Load signals robustly
    nasal = load_signal_txt(nasal_path)
    thor = load_signal_txt(thor_path)
    spo2 = load_signal_txt(spo2_path)

    # Load events robustly
    events = load_events(events_path) if events_path.exists() else None

    # Align thor + spo2 to nasal timestamps
    base_idx = nasal.index
    thor_r = thor.reindex(base_idx, method="nearest")
    spo2_r = spo2.reindex(base_idx, method="nearest")

    # Output PDF
    if out_pdf_path is None:
        out_pdf_path = p / f"{p.name}_visualization.pdf"

    with PdfPages(out_pdf_path) as pdf:
        plt.figure(figsize=(16, 10))

        ax1 = plt.subplot(3, 1, 1)
        ax1.plot(base_idx, nasal, label="Nasal Airflow")
        ax1.set_ylabel("Airflow")
        if events is not None:
            for _, r in events.iterrows():
                ax1.axvspan(r.start_time, r.end_time, alpha=0.3)
        ax1.legend()

        ax2 = plt.subplot(3, 1, 2, sharex=ax1)
        ax2.plot(base_idx, thor_r, label="Thoracic Movement")
        ax2.set_ylabel("Thorax")
        if events is not None:
            for _, r in events.iterrows():
                ax2.axvspan(r.start_time, r.end_time, alpha=0.3)

        ax3 = plt.subplot(3, 1, 3, sharex=ax1)
        ax3.plot(base_idx, spo2_r, label="SpO₂")
        ax3.set_ylabel("SpO₂ (%)")
        ax3.set_xlabel("Time")
        if events is not None:
            for _, r in events.iterrows():
                ax3.axvspan(r.start_time, r.end_time, alpha=0.3)

        plt.tight_layout()
        pdf.savefig()
        if show:
            plt.show()
        plt.close()

    print(f"Visualization saved to: {out_pdf_path}")
    return out_pdf_path


# Example:
# visualize_participant('/content/drive/MyDrive/projects/Health_Sensing/Data/AP01')


In [18]:

def bandpass_butter_lowlevel(signal, fs, lowcut=0.08, highcut=0.6, order=4):
    """Design and apply a Butterworth bandpass filter using second-order sections (stable).
    Default lowcut/highcut chosen to retain typical breathing frequencies (0.08-0.6 Hz).
    fs: sampling frequency in Hz
    Returns filtered signal (pandas Series if input was Series)
    """
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    x = signal.values if isinstance(signal, pd.Series) else np.asarray(signal)
    y = sosfiltfilt(sos, x)
    if isinstance(signal, pd.Series):
        return pd.Series(y, index=signal.index)
    else:
        return y

# Example usage:
# fs = 32
# nasal_f = bandpass_butter_lowlevel(nasal, fs)


In [19]:

def sliding_windows_labeling(nasal, thor, spo2, events_df, window_s=30, overlap=0.5, fs=32):
    """Split signals into windows of window_s seconds with overlap and assign labels.
    events_df is expected to contain columns ['start_time','end_time','event_type'] with datetime type.
    Returns a DataFrame with columns: ['start', 'end', 'label', 'nasal', 'thor', 'spo2']
    where nasal/thor/spo2 are numpy arrays for the window.
    """
    step = int(window_s * (1 - overlap) * fs)
    win_len = int(window_s * fs)

    # Use nasal index as time grid and convert to numeric seconds relative to start
    t0 = nasal.index[0]
    total_samples = len(nasal)

    windows = []
    for start_idx in range(0, total_samples - win_len + 1, step):
        end_idx = start_idx + win_len
        start_time = nasal.index[start_idx]
        end_time = nasal.index[end_idx-1]

        # Determine overlap with events
        label = 'Normal'
        if events_df is not None and len(events_df)>0:
            # compute overlap duration for each event
            overlaps = []
            for _, ev in events_df.iterrows():
                s = ev['start_time']
                e = ev['end_time']
                # intersection
                inter_start = max(s, start_time)
                inter_end = min(e, end_time)
                overlap_seconds = max((inter_end - inter_start).total_seconds(), 0)
                overlaps.append((overlap_seconds, ev.get('event_type','Event')))
            # if any event overlaps more than 50% of window, assign that label
            max_overlap, ev_type = max(overlaps, key=lambda x: x[0]) if overlaps else (0, None)
            if max_overlap >= 0.5*window_s:
                label = ev_type

        nasal_win = nasal.iloc[start_idx:end_idx].values
        thor_win = thor.iloc[start_idx:end_idx].values if len(thor)>=end_idx else np.pad(thor.iloc[start_idx:end_idx].values, (0, max(0, win_len - len(thor.iloc[start_idx:end_idx].values))))
        spo2_win = spo2.iloc[start_idx:end_idx*int(4/32) if False else start_idx:end_idx].values if False else spo2.reindex(nasal.index).iloc[start_idx:end_idx].values

        windows.append({'start':start_time, 'end':end_time, 'label':label,
                        'nasal':nasal_win, 'thor':thor_win, 'spo2':spo2_win})
    dfw = pd.DataFrame(windows)
    return dfw

# Note: the function above assumes signals were resampled/reindexed to the same index (use reindex as shown in visualize_participant)


In [20]:

def save_dataset_windows(df_windows, out_path):
    """Save the windows DataFrame to a parquet file. We convert the numpy arrays to lists
    to make them serializable in parquet.
    """
    df = df_windows.copy()
    df['nasal'] = df['nasal'].apply(lambda x: x.tolist())
    df['thor'] = df['thor'].apply(lambda x: x.tolist())
    df['spo2'] = df['spo2'].apply(lambda x: x.tolist())
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(out_path, index=False)
    print(f"Saved dataset to {out_path}")

# Example usage:
# save_dataset_windows(dfw, 'Dataset/AP01_windows.parquet')


In [21]:

class BreathingWindowDataset(Dataset):
    def __init__(self, df_windows, transform=None):
        self.df = df_windows.reset_index(drop=True)
        self.transform = transform
        self.label_map = {'Normal':0, 'Hypopnea':1, 'Obstructive Apnea':2}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        x_nasal = np.array(row['nasal'], dtype=np.float32)
        x_thor = np.array(row['thor'], dtype=np.float32)
        # Stack channels: nasal and thor -> shape (channels, timesteps)
        x = np.stack([x_nasal, x_thor], axis=0)
        y = self.label_map.get(row['label'], 0)
        return x, y


In [22]:

class Simple1DCNN(nn.Module):
    def __init__(self, in_channels=2, n_classes=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 16, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):
        return self.net(x)

# Example instantiation
model = Simple1DCNN()


In [23]:

class ConvLSTMBlock(nn.Module):
    def __init__(self, in_channels=2, hidden_dim=64, n_classes=3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        # LSTM expects (seq_len, batch, features), so we transpose accordingly
        self.lstm = nn.LSTM(input_size=32, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        # x: (batch, channels, timesteps)
        c = self.conv(x)  # (batch, filters, t')
        c = c.permute(0, 2, 1)  # (batch, t', features)
        out, _ = self.lstm(c)
        out = out[:, -1, :]
        return self.fc(out)

# model = ConvLSTMBlock()


In [24]:

def loso_train_and_evaluate(all_windows_df, participant_column='participant', epochs=10, batch_size=32, device='cpu'):
    """Perform Leave-One-Subject-Out CV. all_windows_df must contain a column with participant ids.
    This function is a skeleton: it trains a model for each fold and collects metrics.
    """
    participants = all_windows_df[participant_column].unique()
    results = {}
    for test_p in participants:
        print('Fold - test participant:', test_p)
        train_df = all_windows_df[all_windows_df[participant_column]!=test_p]
        test_df = all_windows_df[all_windows_df[participant_column]==test_p]

        train_ds = BreathingWindowDataset(train_df)
        test_ds = BreathingWindowDataset(test_df)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

        model = Simple1DCNN()
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        opt = optim.Adam(model.parameters(), lr=1e-3)

        # training loop (small number of epochs here)
        for ep in range(epochs):
            model.train()
            for xb, yb in train_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                opt.zero_grad()
                loss.backward()
                opt.step()

        # evaluation
        model.eval()
        y_true = []
        y_pred = []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(device)
                out = model(xb)
                preds = out.argmax(dim=1).cpu().numpy()
                y_pred.extend(preds.tolist())
                y_true.extend(yb.numpy().tolist())

        cm = confusion_matrix(y_true, y_pred)
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average=None, zero_division=0)
        rec = recall_score(y_true, y_pred, average=None, zero_division=0)

        results[test_p] = {'confusion_matrix': cm, 'accuracy': acc, 'precision': prec.tolist(), 'recall': rec.tolist()}

    return results

# NOTE: Running LOSO with actual training can take time. Execute interactively with smaller epochs while testing.


In [25]:
# Assuming your data is in '/content/drive/MyDrive/projects/Health_Sensing/Data/'
# Adjust this path if your data is located elsewhere
data_root = Path('/content/drive/MyDrive/projects/Health_Sensing/Data/')

all_participant_windows = []
fs = 32 # Sampling frequency, based on examples in bandpass_butter_lowlevel

# Loop through each participant directory to load, preprocess, and window their data
for participant_dir in data_root.iterdir():
    # Check if it's a directory and starts with 'AP' (e.g., AP01, AP02)
    if participant_dir.is_dir() and participant_dir.name.startswith('AP'):
        print(f"Processing participant: {participant_dir.name}")

        # Define paths to signal and event files based on the structure in visualize_participant
        nasal_path = participant_dir / 'Flow-30-05-2024.txt'
        thor_path  = participant_dir / 'Thorac-30-05-2024.txt'
        spo2_path  = participant_dir / 'SPO2-30-05-2024.txt'
        events_path = participant_dir / 'FlowEvents-30-05-2024.txt'

        # Skip if essential signal files are missing
        if not (nasal_path.exists() and thor_path.exists() and spo2_path.exists()):
            print(f"Skipping {participant_dir.name}: Missing essential signal files. Ensure paths are correct.")
            continue

        try:
            # 1. Load signals using the robust loader
            nasal = load_signal_txt(nasal_path, time_col=0, value_col=1)
            thor = load_signal_txt(thor_path, time_col=0, value_col=1)
            spo2 = load_signal_txt(spo2_path, time_col=0, value_col=1)

            # 2. Load events (optional, handles if file doesn't exist)
            events = load_events(events_path) if events_path.exists() else None

            # 3. Align thoracic and SpO2 signals to the nasal airflow's timestamp index
            # This ensures all signals are on the same time grid for windowing
            base_idx = nasal.index
            thor_aligned = thor.reindex(base_idx, method="nearest")
            spo2_aligned = spo2.reindex(base_idx, method="nearest")

            # 4. Apply bandpass filter to breathing-related signals (nasal and thoracic)
            # SpO2 is typically not bandpass filtered for breathing rhythms
            nasal_filtered = bandpass_butter_lowlevel(nasal, fs=fs)
            thor_filtered = bandpass_butter_lowlevel(thor_aligned, fs=fs)
            spo2_final = spo2_aligned # Use the aligned SpO2 directly

            # 5. Create 30-second windows with 50% overlap
            participant_windows_df = sliding_windows_labeling(
                nasal=nasal_filtered,
                thor=thor_filtered,
                spo2=spo2_final,
                events_df=events,
                window_s=30,
                overlap=0.5,
                fs=fs
            )

            # Add a 'participant' column to identify the source of each window
            participant_windows_df['participant'] = participant_dir.name
            all_participant_windows.append(participant_windows_df)

        except Exception as e:
            print(f"Error processing participant {participant_dir.name}: {e}")
            continue

# Check if any data was processed
if not all_participant_windows:
    print("No participant data was processed. Please check your data directory structure and file names.")
else:
    # 6. Concatenate all participant windows into a single DataFrame
    combined_windows_df = pd.concat(all_participant_windows, ignore_index=True)
    print(f"\nTotal windows generated across all participants: {len(combined_windows_df)}")
    print(f"Combined DataFrame head:\n{combined_windows_df.head()}")

    # 7. Evaluate the model using Leave-One-Subject-Out (LOSO) cross-validation
    print("\nStarting Leave-One-Subject-Out (LOSO) cross-validation...")

    # Determine device for training (GPU if available, otherwise CPU)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Run LOSO evaluation. Note: 'epochs' is set low for quick demonstration.
    # For robust training and evaluation, you would typically increase the number of epochs.
    loso_results = loso_train_and_evaluate(
        combined_windows_df,
        participant_column='participant',
        epochs=5, # Consider increasing this for actual training (e.g., 50-100)
        batch_size=32,
        device=device
    )

    print("\nLOSO Evaluation Results:")
    for participant, metrics in loso_results.items():
        print(f"-- Participant {participant} (test set) --")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        # Precision and Recall are provided per class (Normal, Hypopnea, Obstructive Apnea)
        # Ensure your dataset contains these labels if you expect non-zero scores.
        print(f"  Precision (Normal, Hypopnea, Obstructive Apnea): {metrics['precision']}")
        print(f"  Recall (Normal, Hypopnea, Obstructive Apnea): {metrics['recall']}")
        print(f"  Confusion Matrix:\n{metrics['confusion_matrix']}")
        print("-" * 40)

    # Calculate and print average accuracy across all folds
    if loso_results:
        avg_accuracy = np.mean([res['accuracy'] for res in loso_results.values()])
        print(f"\nAverage Accuracy across all LOSO folds: {avg_accuracy:.4f}")
    else:
        print("No results to aggregate.")


Processing participant: AP04
Skipping AP04: Missing essential signal files. Ensure paths are correct.
Processing participant: AP02
Skipping AP02: Missing essential signal files. Ensure paths are correct.
Processing participant: AP05
Skipping AP05: Missing essential signal files. Ensure paths are correct.
Processing participant: AP03
Skipping AP03: Missing essential signal files. Ensure paths are correct.
Processing participant: AP01


/tmp/ipython-input-3440653294.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  index = pd.to_datetime(time_data, errors="coerce")
/tmp/ipython-input-3440653294.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  index = pd.to_datetime(time_data, errors="coerce")


Error processing participant AP01: The length of the input vector x must be greater than padlen, which is 27.
No participant data was processed. Please check your data directory structure and file names.


/tmp/ipython-input-3440653294.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  index = pd.to_datetime(time_data, errors="coerce")
/tmp/ipython-input-3440653294.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
/tmp/ipython-input-3440653294.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")



## How to use this notebook

1. Place participant folders under `Data/` following the naming convention (AP01, AP02, ...).
2. Run the *Imports* cell.
3. Use `visualize_participant('Data/AP01')` to generate PDFs for individual participants.
4. Use the filtering functions to preprocess signals before windowing (e.g., `bandpass_butter_lowlevel`).
5. Create windows with `sliding_windows_labeling` (make sure to reindex signals to same sampling grid first).
6. Save datasets with `save_dataset_windows` (parquet recommended).
7. Train models using `loso_train_and_evaluate` — adjust `epochs`, `device` and `batch_size` as needed.

---

**Recommendation for saving dataset format:** Parquet is compact, fast to read, and supports columns with lists. It integrates well with pandas and PyArrow.

Good luck — run cells interactively and adapt paths & column names to your exact data files.
